# Add Dependences

In [38]:
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")
include("../../src/struct_data.jl")

ModelParameters

# Add Data

In [39]:
parameters = ModelParameters(
    Force = Dict(
        "μ"      => 0.00003, 
        "r_max"  => 3.1,
        "r_min"  => 2.0,
    ),
    Contractile = Dict(
        "f_p"    => 0.00000001
    ),
    Time = Dict(
        "T_f" => 100000,
        "dt"  => 0.1
    ),
    Neighbor = Dict(
        "n_knn" => 50,
        "nn"    => 12
    ),
    Geometry = Dict(
        "R_agg"   => 9,
        "num_agg" => 2
    ),
    Simulation  = Dict(
        "n_text" => 200
    )
);

In [40]:
R_agg = parameters.Geometry["R_agg"]
X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

## Var before func
@select_force cubic
force = cubic(μ = parameters.Force["μ"], r_max = parameters.Force["r_max"], r_min = parameters.Force["r_min"])

cubic(3.0e-5, 3.1, 2.0, var"#142#144"{Float64, Float64, Float64}(3.0e-5, 3.1, 2.0))

## forces.jl -> cu_forces

In [41]:
function cui_force(t, force, fp, nnn)
    # Definig Variables for calculing dX
    global X; global dX; global idx

    # Calculating distance for random forces (contractile)
    r_p = X - X[rand_idx[mod(t, size(X,1))+1, :], :]
    # Finding Distances/Norm for random forces
    dist_p = (sum(r_p .^ 2, dims=2).^ 0.5)

    # Finding distances
    r = reshape(repeat(X, inner=(nnn,1)), nnn, size(X)[1], 3) - X[getindex.(idx,1),:]

    # Finding Distances(Norm)
    dist = ((sum(r .^ 2, dims=3)) .^ 0.5)[:,:,1]

    # Finding forces for each cell
    F = force.f(dist) .* r ./ dist

    # Calculating de dX   -> dX[i,:] +=  r/dist * F
    dX = sum(F[2:end,:,:]; dims=1)[1,:,:] - fp .* (r_p ./ dist_p)
    synchronize()
end


cui_force (generic function with 1 method)

## run_event.jl -> simulate()

In [42]:
function simulate(PATH, SAVING, n_text,t_f, dt, force, fp, n_knn, nnn)
    # Definig Variables for calculing initial variables
    global X 
    X = X |> cu

    # Inizializate Variables for kNN
    global i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
    global Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
    global idx = hcat([[CartesianIndex(i,1) for i=1:nnn] for j=1:size(X,1)]...) |> cu


    #Inizializate Variable for random force
    global rand_idx = CuArray{Int32}(undef, n_knn, size(X,1))

    # Inizializate Variables for Forces
    global r = zeros(nnn,size(X)[1],3) |> cu
    global r_p = zeros(size(X)) |> cu
    global dist = zeros(nnn, size(X)[1]) |> cu
    global dist_p = zeros(size(X,1)) |> cu
    global F = zeros(nnn, size(X)[1],3) |> cu
    global dX = zeros(size(X)[1],3) |> cu;

    # Calculating position of every cell on the fusion
    p = Progress(Int(t_f/dt),barlen=25)
    for t in 0:Int(t_f/dt)

        # Saving data in a given time (n_text)
        if mod(t, Int(t_f/n_text/dt)) == 0 && SAVING
            X_w = Matrix(X)
            open(PATH, "a") do f
                write(f, "$(size(X, 1))\n")
                write(f, "t=$(t*dt)\n")
                writedlm(f,X_w)
                # writedlm(f,hcat(X_f, X_w), ' ')
            end
        end

        ## LEAPFROG ALGORITHM
        if mod(t,2) == 0
            if mod(t, n_knn) == 0 
                # Calculating kNN
                cu_knn(nnn)
            end

            # Finding index for random forces in a given time (size(X,1))
            if mod(t, 2*size(X,1)) == 0 
                global rand_idx = getindex.(idx, 1)[getindex.(rand(2:nnn,2*size(X,1)) ,1),:]
            end
            # Calculating Forces
            cui_force(t, force, fp, nnn)
        else
            global X = X + dX*dt
        end

        next!(p)
    end
end

simulate (generic function with 1 method)

In [43]:
time = [parameters.Time[i] for i in ["T_f","dt"]]
n_text = parameters.Simulation["n_text"]
fp = parameters.Contractile["f_p"]
neighbor = [parameters.Neighbor[i] for i in ["n_knn","nn"]]
simulate("AAA.xyz", true, n_text,time..., force, fp, neighbor...)

Progress:   4%|█                        |  ETA: 0:28:29m:19error in running finalizer: InterruptException()
maybe_collect at /buildworker/worker/package_linux64/build/src/julia_threads.h:313 [inlined]
jl_gc_pool_alloc at /buildworker/worker/package_linux64/build/src/gc.c:1204
default_stream at /home/nicomosty/.julia/packages/CUDA/DfvRa/lib/cudadrv/stream.jl:32 [inlined]
unsafe_finalize! at /home/nicomosty/.julia/packages/CUDA/DfvRa/src/array.jl:99
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2237 [inlined]
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2419
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1703 [inlined]
run_finalizer at /buildworker/worker/package_linux64/build/src/gc.c:278
jl_gc_run_finalizers_in_list at /buildworker/worker/package_linux64/build/src/gc.c:365
run_finalizers at /buildworker/worker/package_linux64/build/src/gc.c:394
jl_gc_collect at /buildworker/worker/package_linux64/build/src/gc.c:3260
gc at 